# Data cleanup and preparation for analysis

In [3]:
import pandas as pd

# Set random seed for reproducibility.
RANDOM_SEED = 12345

def is_any_value_negative(row):
    for value in row.items():
        try:
            if float(value) < 0:
                return True
        except:
            continue
    return False

# Read CSV file.
df = pd.read_csv('city_day.csv')
print(f"Row count before cleanup: {len(df)}")

# Remove rows with negative values.
df = df[df.apply(lambda row: not is_any_value_negative(row), axis=1)]
print(f"Row count after removing rows with negative values: {len(df)}")

# Remove rows with missing AQI.
df = df[df['AQI'].notna()]
print(f"Row count after removing rows without AQI values: {len(df)}")

# Save to CSV file.
df.to_csv('city_day_cleaned.csv', index=False)

# Shuffle rows.
df = df.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

# Remove columns that are not needed.
columns_to_drop = ['PM2.5','PM10', 'NH3', 'CO', 'SO2', 'O3', 'NOx', 'AQI', 'NO2', 'NO', 'Benzene', 'Toluene', 'Xylene']
df = df.drop(columns_to_drop, axis=1)

# Split into training and test data.
training_df = df.sample(frac=0.75, random_state=RANDOM_SEED)
testing_df = df.drop(training_df.index)

# Save to CSV files.
training_df.to_csv('city_day_cleaned_no_measurements_training.csv', index=False)
testing_df.to_csv('city_day_cleaned_no_measurements_testing.csv', index=False)


# Add feature: Split date into weekday, month and year.
training_df.Date = pd.to_datetime(training_df.Date)
training_df["Weekday"] = training_df.Date.dt.weekday
training_df["Month"] = training_df.Date.dt.month
training_df["Year"] = training_df.Date.dt.year

testing_df.Date = pd.to_datetime(testing_df.Date)
testing_df["Weekday"] = testing_df.Date.dt.weekday
testing_df["Month"] = testing_df.Date.dt.month
testing_df["Year"] = testing_df.Date.dt.year

# Add feature: create a column for each city.
for city in set(training_df.City.values):
    training_df[city] = [0]*training_df.shape[0]
    testing_df[city] = [0]*testing_df.shape[0]
    training_df.loc[training_df.City == city, city] = 1
    testing_df.loc[testing_df.City == city, city] = 1

# Remove remaining columns that are not needed.
columns_to_drop = ['City', 'Date']
training_df = training_df.drop(columns_to_drop, axis=1)
testing_df = testing_df.drop(columns_to_drop, axis=1)


# Save to CSV files.
training_df.to_csv('city_day_cleaned_no_measurements_with_features_training.csv', index=False)
testing_df.to_csv('city_day_cleaned_no_measurements_with_features_testing.csv', index=False)
df.head()

Row count before cleanup: 29531
Row count after removing rows with negative values: 29531
Row count after removing rows without AQI values: 24850


,City,Date,AQI_Bucket
0,Bengaluru,2018-05-18,Satisfactory
1,Lucknow,2017-12-18,Very Poor
2,Bhopal,2020-05-15,Moderate
3,Chandigarh,2019-09-15,Satisfactory
4,Gurugram,2016-02-19,Very Poor
